# VERSION 2

In [1]:
!pip install datasets
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.2 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import numpy as np
import cohere

In [12]:
lang = "simple" #simple dataset for English
top_k = 5

The next step can take a while based on max_docs :(

In [30]:
co = cohere.Client('QvANkrXVSC2CB8hskBoehRdL2MclWXZMdnblVG2b') #pls dont dox me either :'/

max_docs = 100000
docs_stream = load_dataset(f"Cohere/wikipedia-2023-11-embed-multilingual-v3", lang, split="train", streaming=True)

docs = []
doc_embeddings = []

for doc in docs_stream:
    docs.append(doc)
    doc_embeddings.append(doc['emb'])
    if len(docs) >= max_docs:
        break

doc_embeddings = np.asarray(doc_embeddings)

But this step is hella fast :)

In [31]:
queries = [
    "What is the capital of France?",
    "Who wrote the play 'Hamlet'?",
    "What did Petra van Staveren win a gold medal for?",
    "How did Steve Jobs die?",
]

output = []

for query in queries:
    response = co.embed(texts=[query], model='embed-multilingual-v3.0', input_type="search_query")
    query_embedding = response.embeddings
    query_embedding = np.asarray(query_embedding)

    # Compute dot score between query embedding and document embeddings
    dot_scores = np.matmul(query_embedding, doc_embeddings.transpose())[0]
    top_k_hits = np.argpartition(dot_scores, -top_k)[-top_k:].tolist()

    # Sort top_k_hits by dot score
    top_k_hits.sort(key=lambda x: dot_scores[x], reverse=True)

    results = []
    for doc_id in top_k_hits:
        passage = f"{docs[doc_id]['title']}. {docs[doc_id]['text']}"
        distance = dot_scores[doc_id]
        results.append({'passage': passage,
                        'distance': distance})

    output.append({'query': query,
                   'results': results})

In [32]:
print(output)

[{'query': 'What is the capital of France?', 'results': [{'passage': 'Paris. Paris (nicknamed the "City of light") is the capital city of France, and the largest city in France. The area is , and around 2.15 million people live there. If suburbs are counted, the population of the Paris area rises to 10.7 million people. It is the most densely populated city in the European Union, with  20.653 people per square kilometer.', 'distance': 0.6999477005530508}, {'passage': 'Île-de-France. Île-de-France is a region of France. The capital city is Paris. It is also the capital city of France. In 2013 about 12 million people lived in the region. About 2.1 million people live in the city of Paris.', 'distance': 0.6690663924949243}, {'passage': "Paris. As the Roman Empire began to fall apart in the West, the Germanic tribe called the Franks moved in, taking it in 464. In 507, their king Clovis I made it his capital. Charlemagne moved his capital to Aachen in Germany, but Paris continued as an impo

In [34]:
import json

with open('retrieved_passages_cohere.json', 'w') as f:
    json.dump(output, f, indent=4)

# ALTERNATE VERSION:

Just some expiriments I did with the wikipedia retriever, does not work now but can be modified. Should work great but uses scraped wikipedia pages using Beautifulsoup instead of a dataset, which means all the retrieved documents are inconsistent (for example only the first few paragraphs of a Wikipedia page). Just ignore this for now :).

In [ ]:
!pip install -U wikipedia
!pip install -U langchain-community
!pip install wiki-passage-retriever

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=c70f22d7ce995d1ccfd1a5b46ed400dc4c06967211ae743b77df47c1f8c99f7e
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstal

In [ ]:
from langchain_community.retrievers import WikipediaRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

from wiki_passage_retriever.retrieve import get_most_relevant_passages

import spacy
from spacy.tokenizer import Tokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
queries = [
    "What is the capital of France?",
    "Who wrote the play 'Hamlet'?",
    "What did Petra van Staveren win a gold medal for?",
    "How did Steve Jobs die?",
]

In [ ]:
nlp = spacy.load('en_core_web_sm')
tokenizer = nlp.tokenizer

retriever = WikipediaRetriever()

passages = {}

for query in queries:
    query = tokenizer(query)
    doc = nlp(query)
    for ent in doc.ents:
        passages[query] = {}
        docs = retriever.invoke(str(ent))
        for i in range(len(docs)):
            title = docs[i].metadata['title']
            text = docs[i].page_content
            passages[query][title] = text.split("/n/n")
print(passages)

{What is the capital of France?: {'France': ["France, officially the French Republic, is a country located primarily in Western Europe. Its overseas regions and territories include French Guiana in South America, Saint Pierre and Miquelon in the North Atlantic, the French West Indies, and many islands in Oceania and the Indian Ocean, giving it one of the largest discontiguous exclusive economic zones in the world. Metropolitan France shares borders with Belgium and Luxembourg to the north, Germany to the northeast, Switzerland to the east, Italy and Monaco to the southeast, Andorra and Spain to the south, and a maritime border with the United Kingdom to the northwest. Its metropolitan area extends from the Rhine to the Atlantic Ocean and from the Mediterranean Sea to the English Channel and the North Sea. Its eighteen integral regions (five of which are overseas) span a combined area of 643,801 km2 (248,573 sq mi) and have a total population of 68.4 million as of January 2024. France i

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [ ]:
n = 0
i = 0
keys = list(passages.keys())
entity_keys = list(passages[keys[n]].keys())

for passage in (passages[keys[n]][entity_keys[i]]):
    print(passage)

France, officially the French Republic, is a country located primarily in Western Europe. Its overseas regions and territories include French Guiana in South America, Saint Pierre and Miquelon in the North Atlantic, the French West Indies, and many islands in Oceania and the Indian Ocean, giving it one of the largest discontiguous exclusive economic zones in the world. Metropolitan France shares borders with Belgium and Luxembourg to the north, Germany to the northeast, Switzerland to the east, Italy and Monaco to the southeast, Andorra and Spain to the south, and a maritime border with the United Kingdom to the northwest. Its metropolitan area extends from the Rhine to the Atlantic Ocean and from the Mediterranean Sea to the English Channel and the North Sea. Its eighteen integral regions (five of which are overseas) span a combined area of 643,801 km2 (248,573 sq mi) and have a total population of 68.4 million as of January 2024. France is a semi-presidential republic with its capita

In [ ]:
import time
from nltk.tokenize import sent_tokenize

In [ ]:
doc_embeddings = []
passages_test = []

for query in passages:
    for ent in passages[query]:
        for text in passages[query][ent]:
            paragraphs = sent_tokenize(text)
            for passage in paragraphs:
                passages_test.append(passage.strip())
                # create doc_embbeddings using cohere:
                embs = co.embed(texts=[passage], model='embed-multilingual-v3.0', input_type="search_document")
                doc_embeddings.append(embs.embeddings)
                time.sleep(.5)

doc_embeddings = np.asarray(doc_embeddings)

doc_embeddings = doc_embeddings.reshape(doc_embeddings.shape[0], doc_embeddings.shape[2])

print(doc_embeddings)
print(passages_test[0])

[[ 0.03649902  0.04437256 -0.01615906 ... -0.00693512  0.05297852
   0.0231781 ]
 [ 0.01878357  0.01704407 -0.03839111 ... -0.00423813  0.04248047
   0.01931763]
 [ 0.0164032   0.0310669  -0.03289795 ...  0.0335083   0.00732422
   0.01004791]
 ...
 [ 0.01049042  0.0134964  -0.01540375 ...  0.03747559  0.01432037
   0.02098083]
 [ 0.00516891  0.05148315 -0.03573608 ...  0.01565552  0.01498413
  -0.00985718]
 [ 0.00354576  0.03131104 -0.00556564 ...  0.03527832  0.00279236
  -0.0091095 ]]
France, officially the French Republic, is a country located primarily in Western Europe. Its overseas regions and territories include French Guiana in South America, Saint Pierre and Miquelon in the North Atlantic, the French West Indies, and many islands in Oceania and the Indian Ocean, giving it one of the largest discontiguous exclusive economic zones in the world. Metropolitan France shares borders with Belgium and Luxembourg to the north, Germany to the northeast, Switzerland to the east, Italy an